In [ ]:
!kaggle datasets download -d grouplens/movielens-latest-small

Dataset URL: https://www.kaggle.com/datasets/grouplens/movielens-latest-small
License(s): other
  0% 0.00/971k [00:00<?, ?B/s]
100% 971k/971k [00:00<00:00, 49.1MB/s]


In [ ]:
!unzip /content/movielens-latest-small.zip

Archive:  /content/movielens-latest-small.zip
  inflating: README.md               
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357289 sha256=093a2165db88b3185aa9a6aebb1297530e6124672f032fd2a32fe7d603e72014
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


#Коллаборативный фильтр


In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD, accuracy
from surprise.model_selection import cross_validate

In [ ]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
algo = SVD()

algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
accuracy.rmse(predictions)

RMSE: 0.8780


0.8779800985512336

In [ ]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8711  0.8777  0.8729  0.8692  0.8738  0.8729  0.0029  
MAE (testset)     0.6715  0.6751  0.6697  0.6668  0.6701  0.6706  0.0027  
Fit time          1.43    1.44    1.43    2.26    1.47    1.61    0.33    
Test time         0.11    0.23    0.11    0.25    0.12    0.16    0.06    


{'test_rmse': array([0.87111913, 0.87769469, 0.87288794, 0.86919686, 0.87380532]),
 'test_mae': array([0.67153169, 0.67507935, 0.66974156, 0.66675457, 0.67007254]),
 'fit_time': (1.427445888519287,
  1.4420006275177002,
  1.4277186393737793,
  2.2615537643432617,
  1.4724452495574951),
 'test_time': (0.11358118057250977,
  0.22536253929138184,
  0.1121821403503418,
  0.25077295303344727,
  0.11999917030334473)}

In [ ]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movie_id_to_title = dict(zip(movies['movieId'], movies['title']))

In [ ]:
user_id = 107 #107


all_movie_ids = ratings['movieId'].unique()

watched_movies = ratings[ratings['userId'] == user_id]
print("Просмотренные фильмы:")
watched_movies

Просмотренные фильмы:


,userId,movieId,rating,timestamp
16970,107,1,4.0,829322340
16971,107,2,5.0,829322340
16972,107,5,4.0,829322340
16973,107,11,5.0,829322340
16974,107,62,5.0,829322340
16975,107,104,3.0,829322340
16976,107,105,4.0,829322340
16977,107,110,5.0,832003811
16978,107,111,4.0,832003891
16979,107,140,4.0,829322340


In [ ]:
watched_movies = watched_movies['movieId'].tolist()

In [ ]:

unwatched_movies = [movie for movie in all_movie_ids if movie not in watched_movies]

predictions = [algo.predict(user_id, movie_id) for movie_id in unwatched_movies]


top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

print("Топ-10 рекомендаций для пользователя:")
for pred in top_recommendations:
    movie_title = movie_id_to_title.get(pred.iid, "Название не найдено")
    print(f'Фильм: {movie_title} с предсказанным рейтингом {pred.est}')


Топ-10 рекомендаций для пользователя:
Фильм: Schindler's List (1993) с предсказанным рейтингом 4.8372421751742785
Фильм: Princess Bride, The (1987) с предсказанным рейтингом 4.782016104461494
Фильм: Seventh Seal, The (Sjunde inseglet, Det) (1957) с предсказанным рейтингом 4.723307307275787
Фильм: Paths of Glory (1957) с предсказанным рейтингом 4.704555916626845
Фильм: Boondock Saints, The (2000) с предсказанным рейтингом 4.6803244051739545
Фильм: Monty Python and the Holy Grail (1975) с предсказанным рейтингом 4.664190936446182
Фильм: One Flew Over the Cuckoo's Nest (1975) с предсказанным рейтингом 4.6539135639815985
Фильм: Rosemary's Baby (1968) с предсказанным рейтингом 4.646987165574866
Фильм: Third Man, The (1949) с предсказанным рейтингом 4.644745474764134
Фильм: Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001) с предсказанным рейтингом 4.641124710810481


In [ ]:
from surprise import KNNBasic
algo2 = KNNBasic()

algo2.fit(trainset)
predictions = algo2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9509


0.9509466596450099

In [ ]:
predictions = [algo2.predict(user_id, movie_id) for movie_id in unwatched_movies]


top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

print("Топ-10 рекомендаций для пользователя:")
for pred in top_recommendations:
    movie_title = movie_id_to_title.get(pred.iid, "Название не найдено")
    print(f'Фильм: {movie_title} с предсказанным рейтингом {pred.est}')

Топ-10 рекомендаций для пользователя:
Фильм: The Jinx: The Life and Deaths of Robert Durst (2015) с предсказанным рейтингом 5.0
Фильм: Troll 2 (1990) с предсказанным рейтингом 5.0
Фильм: Under the Sand (2000) с предсказанным рейтингом 5.0
Фильм: Elling (2001) с предсказанным рейтингом 5.0
Фильм: Relic, The (1997) с предсказанным рейтингом 5.0
Фильм: It's My Party (1996) с предсказанным рейтингом 5.0
Фильм: Lesson Faust (1994) с предсказанным рейтингом 5.0
Фильм: Assignment, The (1997) с предсказанным рейтингом 5.0
Фильм: Bent (1997) с предсказанным рейтингом 5.0
Фильм: Mephisto (1981) с предсказанным рейтингом 5.0


#Контентный фильтр

In [ ]:
movies['genres_str'] = movies['genres'].apply(lambda x: x.replace('|', ' '))

movies[['movieId', 'title', 'genres_str']].head()


,movieId,title,genres_str
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

genre_matrix = vectorizer.fit_transform(movies['genres_str'])

genre_matrix.shape

(9742, 24)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(genre_matrix)

cosine_sim.shape

(9742, 9742)

In [ ]:
title_to_index = pd.Series(movies.index, index=movies['title']).to_dict()

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = title_to_index.get(title)

    if idx is None:
        print(f"Фильм с названием '{title}' не найден.")
        return []

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return movies['title'].iloc[movie_indices]

In [ ]:
recommended_movies = get_recommendations('Toy Story (1995)')

print("Рекомендованные фильмы:")
for movie in recommended_movies:
    print(movie)

Рекомендованные фильмы:
Antz (1998)
Toy Story 2 (1999)
Adventures of Rocky and Bullwinkle, The (2000)
Emperor's New Groove, The (2000)
Monsters, Inc. (2001)
Wild, The (2006)
Shrek the Third (2007)
Tale of Despereaux, The (2008)
Asterix and the Vikings (Astérix et les Vikings) (2006)
Turbo (2013)
